In [2]:
import pandas as pd

df = pd.read_csv(r"products.csv")
# Keep only rows with Price > 0
df = df.drop_duplicates().reset_index(drop=True)

df.to_csv("filterNewAkakce.csv",index=False)
print(f"Original rows: {len(df)}")

FileNotFoundError: [Errno 2] No such file or directory: 'products.csv'

In [ ]:
import pandas as pd

# CSV dosyasını oku
df = pd.read_csv(r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\filterPhone.csv")
print("İlk 10 satır:")
print(df.head(10))
print("\n" + "=" * 50 + "\n")

# Birleştirilecek kolonlar
columns_to_concat = [
    "storage",
    "ram",
    "phone_brand",
    "phone_model",  
    "dimensions",
    "display_size",
    "display_resolution",
    "os",
    "battery",
    "video",
    "chipset",
    "cpu",
    "gpu",
    "ppi_density",
]

# Yeni DataFrame oluştur
result_df = pd.DataFrame(columns=["productID", "Product Name", "description"])

# Her satır için işlem yap
for index, row in df.iterrows():
    # Boş olmayan değerleri key:value formatında birleştir
    description_parts = []

    for col in columns_to_concat:
        if col in df.columns:  # Kolon varsa
            value = row[col]
            # NaN, None veya boş değilse ekle
            if pd.notna(value) and str(value).strip() != "":
                description_parts.append(f"{col}:{value}")

    # Tüm parçaları birleştir
    description = ";".join(description_parts)

    # Phone model değerini al
    phone_model = (
        row["phone_model"]
        if "phone_model" in df.columns and pd.notna(row["phone_model"])
        else ""
    )

    # Yeni DataFrame'e ekle
    result_df.loc[index] = [
        index + 1,
        phone_model,
        description,
    ]  # Corrected column order

    # İlk 5 sonucu göster (test için)

# CSV olarak kaydet
output_path = (
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\products_descriptions.csv"
)
result_df.to_csv(output_path, index=False, encoding="utf-8")

print(f"\nSonuç CSV dosyası kaydedildi: {output_path}")
print("\nOluşturulan CSV yapısı:")
print(result_df.head())

İlk 10 satır:
  phone_brand              phone_model  price  storage  ram launch_date  \
0       apple      Apple iPhone 16 Pro  77999      256    8   9/20/2024   
1       apple          Apple iPhone 15  55049      256    6   9/22/2023   
2       apple          Apple iPhone 15  64999      512    6   9/22/2023   
3       apple  Apple iPhone 15 Pro Max  94999      512    8   9/22/2023   
4       apple  Apple iPhone 15 Pro Max  94999     1000    8   9/22/2023   
5       apple     Apple iPhone 15 Plus  63749      256    6   9/22/2023   
6       apple     Apple iPhone 15 Plus  70499      512    6   9/22/2023   
7       apple      Apple iPhone 15 Pro  82999      256    8   9/22/2023   
8       apple      Apple iPhone 15 Pro  94545      512    8   9/22/2023   
9       apple      Apple iPhone 15 Pro  73999     1000    8   9/22/2023   

                                      dimensions  weight  \
0  149.6 x 71.5 x 8.3 mm (5.89 x 2.81 x 0.33 in)   199.0   
1  147.6 x 71.6 x 7.8 mm (5.81 x 2.82 x 

In [ ]:
import pandas as pd

input_filename = r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\filterNewAkakce.csv'

df = pd.read_csv(input_filename)

df['Product Name'] = df['Product Name'].str.strip()

df.to_csv(input_filename, index=False)


In [ ]:
import pandas as pd

# CSV dosyasını oku
input_filename = r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\filterNewAkakce.csv'
df = pd.read_csv(input_filename)

# "Fiyatı" kolonu hariç diğer tüm kolon adlarını al
columns_except_fiyat = [col for col in df.columns if col != 'Price']

# Bu kolonlara göre tekrar eden satırları kaldır
unique_df = df.drop_duplicates(subset=columns_except_fiyat)

# Yeni CSV dosyasını kaydet
output_filename = r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\withoutDuplicatePrice.csv'
unique_df.to_csv(output_filename, index=False)

print(f"Fiyatı hariç tekrarsız veri kaydedildi: {output_filename}")


Fiyatı hariç tekrarsız veri kaydedildi: C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\withoutDuplicatePrice.csv


In [4]:
import pandas as pd

# CSV dosyasını oku
input_filename = r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\LstmPriceHistory.csv"
df = pd.read_csv(input_filename)

# İndeksi 1'den başlat
df.index = range(1, len(df) + 1)

# CSV'ye yaz
df.to_csv(input_filename, index=True)

print(len(df))


125648


In [7]:
import pandas as pd


df = pd.read_csv("epeyProductListid.csv")
print("before",len(df))
df = df.drop_duplicates(subset="ProductID")
print("after",len(df))
df.to_csv("epeyProductListid.csv", index=False)


df = pd.read_csv("products.csv")
print("before", len(df))
df =df.drop_duplicates()
print("after", len(df))
df.to_csv("products.csv")

before 342
after 342
before 131531
after 131531
